In [1]:
import numpy as np
import pandas
import tensorflow as tf
from importlib import reload
import graph; reload(graph)
from w266_common import patched_numpy_io

/home/stassyvichev/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# LOAD THE DATA
fileName = "data/encoded_data/embeddings_w266.npy"
encodedData = np.load(fileName)
encodedData = pandas.DataFrame(data = encodedData, columns = ["x_"+str(i) for i in range(500)])
filename = "data/Labeled_Colorado_Flu_Study_Tweets_AvI_RvN_SvO.csv"
coloradoData = pandas.read_csv(filename, sep="\t")
print(encodedData.shape)
allData = pandas.concat([coloradoData, encodedData], axis = 1)
print(allData.shape)

(5270, 500)
(5270, 505)


In [3]:
allData

Tweet_ID                                      Tweet_Content  \
0     5926297601  don't worry it's not swine flu, i already got ...   
1     5222838706  muh. if i am getting sick and it's not swine f...   
2     5233654812  what is up with my boy?  this morning i though...   
3     5918860304  getting better,no more piggy flu 4 me,it was n...   
4     4631607800  @robbsterr yay for man txting you.. in other n...   
5     6004314210  swine flu has arrived at my daughter's kinderg...   
6     5225053106                       i think im getting flu soon.   
7     5229704707  h1n1 flu is rampant in nagasaki. holy crap. 70...   
8     5233811803  rt @adamwsyx6: getting my seasonal flu shot to...   
9     5900297914  @sihawkings you like christmas then? i'll send...   
10    5922861314  it was a robitussin nighttime cold & flu kind ...   
11    5912340611  @marisamir chemo day, plus getting fitted for ...   
12    5922229214  so yeah.. myheads killin me:/ and i think im g...   
13    5984946706  the swine flu is the only thing calipari is wo...   
14    5222406214  is unsure about all this talk of vaccinations....   
15    6003319713  getting a little fed up with having (insert ra...   
16    5916662502  my flu is not going away!!!!! its not even get...   
17    5926793900  rt @deejaykingmidas: i dont know  is which one...   
18    5234564007  trying to decide whether or not to get the h1n...   
19    5932278302  i think ive just about gotten over this flu, b...   
20    5235612004  working through second kiddo with the flu. yea...   
21    5924620503  oh yay! getting chippy food for dinner. i'm ge...   
22    5985417606  at home. i think i have the stomach flu or som...   
23    5991525204  just getting over swine flu.  it must have bee...   
24    5227919214  my throat has that horrible half sore feeling ...   
25    5234220406  the hosp. called elijah has the swine flu. omg...   
26    5231304213  @sarahshum ugh. i'm getting a flu shot (hopefu...   
27    5986621813  so glad @blakewsmith won't be getting the pig ...   
28    5908424206  is down with a cold. to be clear, i was alread...   
29    5221219912  i get sick all the time. i didn't get a flu sh...   
...          ...                                                ...   
5240  3713360905  @amba12 you're probably not afraid of h1n1 bec...   
5241  4034857010  if you force feed a bird nyquil they shit coug...   
5242  4701018212  lol rt @maxlucado: for the bird flu you need t...   
5243  4174555602  #whateverhappenedto bird flu? swine flu came a...   
5244  4652197012  @nbczoraida no flu shot- i'm too worried for t...   
5245  4086053214  gwen will per drs advice-nervous tho -@princes...   
5246  4533635808  @jackalltimelow im getting over the swine flu ...   
5247  3795135905  just fell asleep and lost two hours of study t...   
5248  4793742107  i'm have a serious case of swine-grind flu i'm...   
5249  3907860007  rt @fizzyduck: bird flu, swine flu now monkey ...   
5250  3978570900  really worried bwt her cough but doc say cuz o...   
5251  3964167300                thinking i might have the swine flu   
5252  4468128511  feels really bad! i hope i&#39;m not getting t...   
5253  3942687614  getting a flu shot right now. will someone ple...   
5254  4442863605  @famousclub listening to sum new tracks, think...   
5255  3973344206  rt @pacothefreak: rt @tbtphotog: looks like ga...   
5256  3988851212  3 days until vegas and i feel like i'm getting...   
5257  3902148500  ha! rt @peggyfrezon @k8fenner @mindyburdette @...   
5258  3991481513  @jedirach thinking about  rachel and hope she ...   
5259  4447450202  @axlynnxcx41 duuuuuude omg me too and im getti...   
5260  4466585806  getting a cold!!! damn......sore throat....cou...   
5261  4470230513  is dr. oz getting a swine flu shot? : sometime...   
5262  4476104711  @rafaelcorrales careful with the tweets - you ...   
5263  4046740113  whatever you do, read this before getting swin...   
5264  4495003013  @ktvbkelsey i'

In [4]:
# get all data with labels present for the column we care about
allDataVal = allData.dropna(subset=["Related_Label"])
print(allDataVal.shape)

# extract X and Y as np arrays, so that we can feed them to tensors. 
x = allDataVal[["x_"+str(i) for i in range(500)]]
y = allDataVal["Related_Label"]
print(x.values.shape)
print(y.values.shape)

(4413, 505)
(4413, 500)
(4413,)


In [5]:
# split into train, test, dev (80%,10%,10%)
np.random.seed(42)
train_x, test_x, dev_x = np.split(x.sample(frac=1), [int(.8*len(x)), int(.9*len(x))])
train_y = y[train_x.index]
test_y = y[test_x.index]
dev_y = y[dev_x.index]

# convert to numpy arrays
train_x, test_x, dev_x, train_y, test_y, dev_y = train_x.values, test_x.values, dev_x.values, train_y.values, test_y.values, dev_y.values 
train_y = train_y.astype(int)
dev_y = dev_y.astype(int)
test_y = test_y.astype(int)
train_x = train_x.astype("float32")
test_x = test_x.astype("float32")
dev_x = dev_x.astype("float32")
print(type(train_x))
print(train_x.shape)
print(dev_x.shape)
print(test_x.shape)
print(train_y)

<class 'numpy.ndarray'>
(3530, 500)
(442, 500)
(441, 500)
[0 0 0 ... 0 1 1]


In [19]:
# setting up model params, model function, and input function
reload(graph)

# Specify model hyperparameters as used by model_fn
model_params = dict(hidden_dims=[75, 75, 75, 75], num_classes=2,
                    lr=0.1, optimizer='adagrad', beta=0.1,dropout_rate = 0.5, maxGradNorm = 1.0)

model = tf.estimator.Estimator(model_fn=graph.classifierModelFn, 
                               params=model_params)

# Training params for input function
train_params = dict(batch_size=32, total_epochs=20, eval_every=2)

# set up input function
train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"x": train_x}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

# Input function for dev set batches. As above, but:
# - Don't randomize order
# - Iterate exactly once (one epoch)
dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"x": dev_x}, y=dev_y,
                    batch_size=128, num_epochs=1, shuffle=False
                )


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp4t3wimxz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2e88071e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# Training the actual model
for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train
    model.train(input_fn=train_input_fn)
    
    # evaluate on dev set
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp4t3wimxz/model.ckpt.
INFO:tensorflow:loss = 956.5633, step = 1
INFO:tensorflow:global_step/sec: 283.888
INFO:tensorflow:loss = 930.5387, step = 101 (0.355 sec)
INFO:tensorflow:global_step/sec: 424.651
INFO:tensorflow:loss = 925.1551, step = 201 (0.234 sec)
INFO:tensorflow:Saving checkpoints for 221 into /tmp/tmp4t3wimxz/model.ckpt.
INFO:tensorflow:Loss for final step: 589.829.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-22-09:27:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4t3wimxz/model.ckpt-221
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INF

INFO:tensorflow:Saving checkpoints for 1547 into /tmp/tmp4t3wimxz/model.ckpt.
INFO:tensorflow:Loss for final step: 589.4137.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-22-09:27:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4t3wimxz/model.ckpt-1547
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-22-09:27:29
INFO:tensorflow:Saving dict for global step 1547: accuracy = 0.74208146, cross_entropy_loss = 0.5564221, global_step = 1547, loss = 3160.1777
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4t3wimxz/model.ckpt-1547
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints 

In [21]:
# evaluate on test set
test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"x": test_x}, y=test_y,
                    batch_size=128, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")

#### END(YOUR CODE) ####
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-22-09:27:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4t3wimxz/model.ckpt-2210
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-22-09:27:44
INFO:tensorflow:Saving dict for global step 2210: accuracy = 0.70068026, cross_entropy_loss = 0.58380044, global_step = 2210, loss = 3175.2373
Accuracy on test set: 70.07%


{'accuracy': 0.70068026,
 'cross_entropy_loss': 0.58380044,
 'global_step': 2210,
 'loss': 3175.2373}

In [ ]:
# do some predicting, so we can see which examples fail:
predictions = list(model.predict(test_input_fn))  # list of dicts
